In this demo, we demonstrate how ptype can be used. The tasks are as follows:

- to run ptype on a data frame, and print a summary of the results. 
- to show possible interactions ptype offers to its users, when a change on the predictions is necessary.

In [ ]:
# Preamble to run notebook in context of source package.
# NBVAL_IGNORE_OUTPUT
import sys
sys.path.insert(0, '../')
!{sys.executable} -m pip install -r ../requirements.txt


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}</style>"))

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcdefaults()

from ptype.Ptype import Ptype
from ptype.utils import evaluate_types
import pandas as pd
import numpy as np

## 1 Using ptype
### 1.a Create a ptype assistant

In [ ]:
ptype = Ptype()

### loading data

In [ ]:
dataset_name = 'auto'
dataset_path = '../data/' + dataset_name + '.csv'

df =  pd.read_csv(dataset_path, sep=',', encoding='ISO-8859-1', dtype=str, header=None, keep_default_na=False, skipinitialspace=True)
print(df.shape)
df.head(5)

### 1.b Run ptype

In [ ]:
ptype.run_inference(df)

### 1.c Report the results

In [ ]:
evaluate_types(dataset_name, ptype)

#### Show the results for all of the columns
We can generate a new dataframe which includes the column type predictions in the header.

In [ ]:
df = ptype.show_results_df()
df.head(20)

To inspect the results in detail, we can generate a human-readable description.

In [ ]:
ptype.show_results()

#### Show the results for the columns with missing data

In [ ]:
column_names = ptype.get_columns_with_missing()
ptype.show_results(column_names)

#### Show the results for the columns with anomalies

In [ ]:
column_names = ptype.get_columns_with_anomalies()
ptype.show_results(column_names)

## 2. User Interactions
- changing the column type predictions,
- changing the anomaly type predictions,
- changing the missing type predictions.


### 2.a Change the column type predictions

In [ ]:
dataset_name = 'data_gov_10151_1'
dataset_path = '../data/' + dataset_name + '.csv'
df =  pd.read_csv(dataset_path, sep=',', encoding='ISO-8859-1', dtype=str, keep_default_na=False, skipinitialspace=True)
print(df.shape)
df.head(2)

In [ ]:
ptype.run_inference(df)
ptype.show_results_df().head(20)

#### checking columns annotated with the gender type

In [ ]:
gender_columns = ptype.get_columns_with_type('gender')
ptype.show_results(gender_columns)

In [ ]:
ptype.change_column_type_annotations(gender_columns, ['string',])
ptype.show_results(gender_columns)

### 2.b Changing the anomaly annotations
Notice that the values of 'T' and 'U' are still annotated as anomalies. We need to update the annotations to fix this.

In [ ]:
ptype.change_anomaly_annotations('Status', ['T', 'U'])
ptype.show_results(gender_columns)

### 2.c Change missing data encodings

In [ ]:
dataset_name = 'auto'
dataset_path = '../data/' + dataset_name + '.csv'
df =  pd.read_csv(dataset_path, sep=',', encoding='ISO-8859-1', dtype=str, header=None, keep_default_na=False, skipinitialspace=True)
print(df.shape)
df.head(2)

In [ ]:
ptype.run_inference(df)

#### checking the columns with missing data

In [ ]:
column_names = ptype.get_columns_with_missing()
ptype.show_results(column_names)

In [ ]:
column = 0
ptype.change_missing_data_annotations(column, ['-1'])
ptype.show_results([column])

### 2.d Merging Different Encodings of Missing Data

In [ ]:
dataset_name = 'mass_6'
dataset_path = '../data/' + dataset_name + '.csv'
df =  pd.read_csv(dataset_path, sep=',', encoding='ISO-8859-1', dtype=str, keep_default_na=False, skipinitialspace=True)
print(df.shape)
df.head(2)

In [ ]:
ptype.run_inference(df)

In [ ]:
column_name = 'LRE Ages 3-5 - Full Incl #'
ptype.show_results([column_name,])

In [ ]:
new_encoding = 'NA'
ptype.merge_missing_data(column_name, new_encoding)
ptype.show_results([column_name,])